## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,95.61,63,13,5.01,few clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,45.55,55,0,3.89,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,74.79,77,19,12.19,few clouds
3,3,Castro,BR,-24.7911,-50.0119,57.87,98,100,3.42,overcast clouds
4,4,Chiang Rai,TH,19.9167,99.9167,79.12,98,40,2.30,moderate rain


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,74.79,77,19,12.19,few clouds
5,5,Gamba,GA,-2.6500,10.0000,70.03,93,70,4.43,broken clouds
22,22,Binzhou,CN,37.3667,118.0167,72.16,87,57,3.67,broken clouds
28,28,Yellowknife,CA,62.4560,-114.3525,70.32,56,40,10.36,scattered clouds
54,54,Chicama,PE,-7.8447,-79.1469,70.57,63,23,8.88,few clouds
69,69,Rio Pardo De Minas,BR,-15.6097,-42.5397,71.64,46,52,10.25,broken clouds
75,75,Penukonda,IN,14.0833,77.5833,72.19,79,100,12.08,overcast clouds
91,91,Vaini,TO,-21.2000,-175.2000,74.55,80,68,21.30,broken clouds
92,92,Requena,ES,39.4883,-1.1004,73.80,53,0,5.14,clear sky
100,100,Jamestown,US,42.0970,-79.2353,73.94,83,100,5.99,overcast clouds


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                76
City                   76
Country                76
Lat                    76
Lng                    76
Max Temp               76
Humidity               76
Cloudiness             76
Wind Speed             76
Current Description    76
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                76
City                   76
Country                76
Lat                    76
Lng                    76
Max Temp               76
Humidity               76
Cloudiness             76
Wind Speed             76
Current Description    76
dtype: int64

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,74.79,few clouds,-23.1203,-134.9692,
5,Gamba,GA,70.03,broken clouds,-2.6500,10.0000,
22,Binzhou,CN,72.16,broken clouds,37.3667,118.0167,
28,Yellowknife,CA,70.32,scattered clouds,62.4560,-114.3525,
54,Chicama,PE,70.57,few clouds,-7.8447,-79.1469,
69,Rio Pardo De Minas,BR,71.64,broken clouds,-15.6097,-42.5397,
75,Penukonda,IN,72.19,overcast clouds,14.0833,77.5833,
91,Vaini,TO,74.55,broken clouds,-21.2000,-175.2000,
92,Requena,ES,73.80,clear sky,39.4883,-1.1004,
100,Jamestown,US,73.94,overcast clouds,42.0970,-79.2353,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
print('scanning...')
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print('done')

scanning...
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
done


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,74.79,few clouds,-23.1203,-134.9692,People ThankYou
5,Gamba,GA,70.03,broken clouds,-2.6500,10.0000,
22,Binzhou,CN,72.16,broken clouds,37.3667,118.0167,Hampton By Hilton Binzhou
28,Yellowknife,CA,70.32,scattered clouds,62.4560,-114.3525,The Explorer Hotel
54,Chicama,PE,70.57,few clouds,-7.8447,-79.1469,Balihai Bungalows
...,...,...,...,...,...,...,...
629,Sri Aman,MY,73.92,overcast clouds,1.2376,111.4621,Seri Simanggang Hotel
636,Axim,GH,74.95,overcast clouds,4.8699,-2.2405,Axim Beach Resort And Spa
644,Raymond,US,74.10,moderate rain,43.9015,-70.4703,Migis Lodge at Sebago Lake
647,Baturaja,ID,73.27,overcast clouds,-4.1333,104.1667,"HOTEL Sekar Sunja, Baturaja Timur, Ogan Komeri..."


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure

locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))